In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2. 利用课堂案例，实现分布式DDP模型训练。存盘后加载实现推理。

In [2]:
# 升级numpy（如果cesium已兼容numpy 2.0+）
!pip install numpy>=2.0

# 降级rich
!pip install rich<14

# 降级所有NVIDIA CUDA库（根据torch 2.6.0+cu124的要求）
!pip install nvidia-cublas-cu12==12.4.5.8 \
            nvidia-cudnn-cu12==9.1.0.70 \
            nvidia-cufft-cu12==11.2.1.3 \
            nvidia-curand-cu12==10.3.5.147 \
            nvidia-cusolver-cu12==11.6.1.9 \
            nvidia-cusparse-cu12==12.3.1.170 \
            nvidia-nvjitlink-cu12==12.4.127

# 升级fsspec
!pip install fsspec==2025.3.2

/bin/bash: line 1: 14: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.10.19:
      Successfully u

In [3]:
!pip -q install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
%%writefile week12_test02_ner_ddp.py

import os
import numpy as np
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

# 清理分布式环境
def cleanup():
    dist.destroy_process_group()
    

def train(rank, world_size):
    setup(rank, world_size)
    # 数据集
    ds = load_dataset('nlhappy/CLUE-NER')
    # entity_index
    entites = ['O'] + ['movie', 'name', 'game', 'address', 'position', \
               'company', 'scene', 'book', 'organization', 'government']
    tags = ['O']
    for entity in entites[1:]:
        tags.append('B-' + entity.upper())
        tags.append('I-' + entity.upper())
    
    entity_index = {entity:i for i, entity in enumerate(entites)}

    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
    
    def entity_tags_proc(item):
        # item即是dataset中记录
        text_len = len(item['text'])  # 根据文本长度生成tags列表
        tags = [0] * text_len    # 初始值为‘O’
        # 遍历实体列表，所有实体类别标记填入tags
        entites = item['ents']
        for ent in entites:
            indices = ent['indices']  # 实体索引
            label = ent['label']   # 实体名
            tags[indices[0]] = entity_index[label] * 2 - 1
            for idx in indices[1:]:
                tags[idx] = entity_index[label] * 2
        return {'ent_tag': tags}
    
    # 使用自定义回调函数处理数据集记录
    ds1 = ds.map(entity_tags_proc)
    
    def data_input_proc(item):
        # 输入文本先拆分为字符，再转换为模型输入的token索引
        batch_texts = [list(text) for text in item['text']]
        # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
        input_data = tokenizer(batch_texts, truncation=True, add_special_tokens=False, max_length=512, 
                               is_split_into_words=True, padding='max_length')
        input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ent_tag']]
        return input_data
        
    
    ds2 = ds1.map(data_input_proc, batched=True)  # batch_size 1000
    
    
    local_rank = rank
    
    id2lbl = {i:tag for i, tag in enumerate(tags)}
    lbl2id = {tag:i for i, tag in enumerate(tags)}
    
    model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                            num_labels=21,
                                                            id2label=id2lbl,
                                                            label2id=lbl2id)
    model.to(local_rank)
    
    args = TrainingArguments(
        output_dir="ner_train02",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
        num_train_epochs = 3,    # 训练 epoch
        save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
        per_device_train_batch_size=16,  # 训练批次
        per_device_eval_batch_size=16,
        report_to='tensorboard',  # 训练输出记录
        eval_strategy="epoch",
        local_rank=local_rank,   # 当前进程 RANK
        fp16=True,               # 使用混合精度
        lr_scheduler_type='linear',  # 动态学习率
        warmup_steps=100,        # 预热步数
        ddp_find_unused_parameters=False  # 优化DDP性能
    )
    
    def compute_metric(result):
        # result 是一个tuple (predicts, labels)
        
        # 获取评估对象
        seqeval = evaluate.load('seqeval')
        predicts,labels = result
        predicts = np.argmax(predicts, axis=2)
        
        # 准备评估数据
        predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        results = seqeval.compute(predictions=predicts, references=labels)
    
        return results
    
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)
    
    trainer = Trainer(
        model,
        args,
        train_dataset=ds2['train'],
        eval_dataset=ds2['validation'],
        data_collator=data_collator,
        compute_metrics=compute_metric
    )
    
    trainer.train()

def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

if __name__ == "__main__":
    main()

Writing week12_test02_ner_ddp.py


In [5]:
!python week12_test02_ner_ddp.py

2025-06-12 15:12:53.032481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749741173.542300      83 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749741173.657185      83 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 15:13:21.111077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-12 15:13:21.112715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:17497

In [6]:
##模型存在目录'ner_train02/checkpoint-1008' 加载
from transformers import pipeline
pipeline=pipeline('token-classification','ner_train02/checkpoint-1008')

2025-06-12 16:18:33.021193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749745113.046806      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749745113.054573      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cuda:0


In [7]:
pipeline("2024年10月15日，马斯克在上海特斯拉工厂宣布了新款Model 3的价格。")

[{'entity': 'B-NAME',
  'score': 0.8173873,
  'index': 9,
  'word': '马',
  'start': 12,
  'end': 13},
 {'entity': 'I-NAME',
  'score': 0.8692206,
  'index': 10,
  'word': '斯',
  'start': 13,
  'end': 14},
 {'entity': 'I-NAME',
  'score': 0.85123503,
  'index': 11,
  'word': '克',
  'start': 14,
  'end': 15},
 {'entity': 'B-ADDRESS',
  'score': 0.86964536,
  'index': 13,
  'word': '上',
  'start': 16,
  'end': 17},
 {'entity': 'I-ADDRESS',
  'score': 0.80002487,
  'index': 14,
  'word': '海',
  'start': 17,
  'end': 18},
 {'entity': 'I-COMPANY',
  'score': 0.46123448,
  'index': 15,
  'word': '特',
  'start': 18,
  'end': 19},
 {'entity': 'I-COMPANY',
  'score': 0.7973602,
  'index': 16,
  'word': '斯',
  'start': 19,
  'end': 20},
 {'entity': 'I-COMPANY',
  'score': 0.8343343,
  'index': 17,
  'word': '拉',
  'start': 20,
  'end': 21},
 {'entity': 'I-COMPANY',
  'score': 0.5760216,
  'index': 18,
  'word': '工',
  'start': 21,
  'end': 22},
 {'entity': 'I-COMPANY',
  'score': 0.5637514,
  'in

In [8]:
pipeline("牛顿发明了微积分，而牛顿苹果是一款知名的电子产品品牌")

[{'entity': 'B-NAME',
  'score': 0.8034442,
  'index': 1,
  'word': '牛',
  'start': 0,
  'end': 1},
 {'entity': 'I-NAME',
  'score': 0.9430462,
  'index': 2,
  'word': '顿',
  'start': 1,
  'end': 2},
 {'entity': 'B-COMPANY',
  'score': 0.574772,
  'index': 11,
  'word': '牛',
  'start': 10,
  'end': 11},
 {'entity': 'I-NAME',
  'score': 0.58373725,
  'index': 12,
  'word': '顿',
  'start': 11,
  'end': 12}]